In [1]:
import VariationalBayes as vb
import VariationalBayes.ExponentialFamilies as ef

import LogisticGLMM_lib as logit_glmm
import VariationalBayes.SparseObjectives as obj_lib

import matplotlib.pyplot as plt
%matplotlib inline

import autograd
import autograd.numpy as np

import copy
from scipy import optimize

import os
import json

import time

import pickle


In [2]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.

#analysis_name = 'simulated_data_small'
analysis_name = 'criteo_subsampled'

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')

pickle_output_filename = os.path.join(data_dir, '%s_python_vb_results.pkl' % analysis_name)

pkl_file = open(pickle_output_filename, 'rb')
vb_results = pickle.load(pkl_file)


In [4]:
json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)
y_g_vec, y_vec, x_mat, glmm_par, prior_par = logit_glmm.load_json_data(json_filename)

K = x_mat.shape[1]
NG = np.max(y_g_vec) + 1

# Define moment parameters
moment_wrapper = logit_glmm.MomentWrapper(glmm_par)
glmm_par = logit_glmm.get_glmm_parameters(K=K, NG=NG)
glmm_par.set_free(vb_results['glmm_par_free'])
model = logit_glmm.LogisticGLMM(
    glmm_par, prior_par, x_mat, y_vec, y_g_vec, vb_results['num_gh_points'])
objective = obj_lib.Objective(model.glmm_par, model.get_kl)
moment_wrapper = logit_glmm.MomentWrapper(glmm_par)
moment_vec = moment_wrapper.get_moment_vector_from_free(vb_results['glmm_par_free'])

print(vb_results.keys())

dict_keys(['tau_prior_alpha', 'beta_prior_info', 'x', 'y', 'y_group', 'NG', 'mu_prior_info', 'mu_prior_mean', 'tau_prior_beta', 'N', 'K', 'beta_prior_mean'])
dict_keys(['cg_row_time', 'kl_hess_packed', 'lrvb_cov', 'moment_jac', 'y_vec', 'x_mat', 'glmm_par_free', 'num_cg_iterations', 'beta_dim', 'run_name', 'vb_prior_sens', 'num_gh_points', 'prior_par_vec', 'y_g_vec', 'inverse_time', 'num_groups', 'glmm_par_vector', 'kl_inv_moment_jac', 'log_prior_hess', 'vb_time', 'hess_time'])


In [5]:
glmm_par_opt = copy.deepcopy(glmm_par)
def tr_optimize(trust_init, prior_par, gtol=1e-6, maxiter=500):
    model.prior_par.set_vector(prior_par.get_vector())
    objective.logger.initialize()
    objective.logger.print_every = 5
    vb_opt = optimize.minimize(
        lambda par: objective.fun_free(par, verbose=True),
        x0=trust_init,
        method='trust-ncg',
        jac=objective.fun_free_grad,
        hessp=objective.fun_free_hvp,
        tol=1e-6, options={'maxiter': maxiter, 'disp': True, 'gtol': gtol })
    return vb_opt.x


In [6]:

perturb_prior_par = 'mu_prior_info'
epsilon_list = np.linspace(0.0, 10.0, 20)
moment_vec_list = []

glmm_opt_pert = vb_results['glmm_par_free']
vb_refit_time = time.time()
for epsilon in epsilon_list:
    print('Epsilon: ', epsilon)
    prior_par_pert = copy.deepcopy(prior_par)
    prior_par_pert[perturb_prior_par].set(prior_par[perturb_prior_par].get() + epsilon)

    glmm_opt_pert = tr_optimize(trust_init=glmm_opt_pert, prior_par=prior_par_pert)
    moment_vec_list.append(moment_wrapper.get_moment_vector_from_free(glmm_opt_pert))
    
vb_refit_time = time.time() - vb_refit_time

Epsilon:  0.0
Iter  0  value:  23825.6016424
         Current function value: 23825.601642
         Iterations: 0
         Function evaluations: 2
         Gradient evaluations: 1
         Hessian evaluations: 0
Epsilon:  0.526315789474
Iter  0  value:  23826.6984672
Iter  5  value:  23826.6860697
Optimization terminated successfully.
         Current function value: 23826.677599
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10
         Hessian evaluations: 0
Epsilon:  1.05263157895
Iter  0  value:  23827.7330912
Iter  5  value:  23827.7210465
Optimization terminated successfully.
         Current function value: 23827.713403
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10
         Hessian evaluations: 0
Epsilon:  1.57894736842
Iter  0  value:  23828.7298932
Iter  5  value:  23828.7181892
Optimization terminated successfully.
         Current function value: 23828.711297
         Iterations: 9
        

In [ ]:
prior_indices = copy.deepcopy(prior_par)
prior_indices.set_vector(np.arange(prior_indices.vector_size()))
mu_info_col = prior_indices[perturb_prior_par].get()[0]

pred_sens_vec = vb_results['vb_prior_sens'][:, mu_info_col]

moment_wrapper.moment_par.set_vector(pred_sens_vec)
mu_pred_sens = moment_wrapper.moment_par['e_mu'].get()[0]

moment_wrapper.moment_par.set_vector(moment_vec)

def GetEMu(moment_vec):
    moment_wrapper.moment_par.set_vector(moment_vec)
    return np.squeeze(moment_wrapper.moment_par['e_mu'].get())
    
e_mu_list = [ GetEMu(moment_vec) for moment_vec in moment_vec_list ]
e_mu_base = e_mu_list[0]

plt.plot(epsilon_list, e_mu_list - e_mu_base, 'k.')
plt.plot(epsilon_list, e_mu_list - e_mu_base, 'k')
plt.plot(epsilon_list, epsilon_list * mu_pred_sens, 'r')

print('Refit time: ', vb_refit_time)

# Note that this includes a "refit" at the zero point -- let's just remove it
# and bias the result slightly against us.
print('Refit time per iteration: ', vb_refit_time / (len(epsilon_list) - 1))

In [ ]:
# Write the result to a JSON file for use in R.

run_name = 'production'

pickle_output_filename = os.path.join(data_dir, '%s_python_vb_perturbation_results.pkl' % analysis_name)
pickle_output = open(pickle_output_filename, 'wb')

# Unlike with JSON, numpy arrays can be pickled.
pickle_result_dict = {  'run_name': run_name,
                        'pickle_output_filename': pickle_output_filename,
                        'vb_refit_time': vb_refit_time,
                        'epsilon_list': epsilon_list,
                        'moment_vec_list': moment_vec_list,
                        'perturb_prior_par': perturb_prior_par
                     }

# Pickle dictionary using protocol 0.
pickle.dump(pickle_result_dict, pickle_output)
pickle_output.close()

print(pickle_output_filename)


print('\n\nDONE.')
